# Wikimedia data
Wikimedia Downloads: Analytics Datasets Info about Pageviews, mediacounts and unique devices:
Pageviews since may 2015:
https://dumps.wikimedia.org/other/pageviews/[YEAR]/[YEAR]-[2-DIGIT-MONTH]/projectviews-YYYYMMDD-HH0000

In [ ]:
# generate URLs for download
from datetime import datetime, timedelta

def time_range(init, end):
    while init<end:
        yield init
        init+=timedelta(hours=1)   
    
def explode_time(date):
    return date, {
        "year": date.year,
        "month": date.month,
        "day": date.day,
        "hour": date.hour,
    }

def gen_url(x):
    ( date, exploded_date ) = x
    string = "https://dumps.wikimedia.org/other/pageviews/{year}/{year}-" + \
             "{month:02d}/projectviews-{year}{month:02d}{day:02d}-{hour:02d}0000"
    return date, string.format(**exploded_date)

exploded_dates = map(explode_time, time_range(datetime(2016,1,1), datetime(2016,1,2)))
urls = map(gen_url, exploded_dates)

#list(urls)

In [ ]:
import requests
import re
from itertools import chain
from functools import reduce

def explode_url_lines(x):
    ( date, url ) = x
    r = requests.get(url)
    for line in r.text.splitlines():
        yield date, line

def filter_wikipedia_en(dateline):
    ( date, line ) = dateline
    return re.match(r"^en -", line)
    
file_generator = map(explode_url_lines, urls)

dateline_generator = reduce(chain, file_generator)

wikipedia_en_generator = filter(filter_wikipedia_en, dateline_generator)

for date,line in wikipedia_en_generator:
    print(date,line)
